In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\Projects\\4.Deep Learning\\End-to-End-Chest-Cancer-Classification-System-with-MLOps-DVC-MLflow-CI-CD-AWS-Deployment-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\Projects\\4.Deep Learning\\End-to-End-Chest-Cancer-Classification-System-with-MLOps-DVC-MLflow-CI-CD-AWS-Deployment-'

In [ ]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mamoonali926568/End-to-End-Chest-Cancer-Classification-System-with-MLOps-DVC-MLflow-CI-CD-AWS-Deployment-.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mamoonali926568"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b34569891d83fb8cb187d03d1310e2bd5e200964"

In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/mamoonali926568/End-to-End-Chest-Cancer-Classification-System-with-MLOps-DVC-MLflow-CI-CD-AWS-Deployment-.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-02-14 12:11:44,691: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-14 12:11:44,700: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-14 12:11:44,704: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 11s 1s/step - loss: 3.7296e-04 - accuracy: 1.0000
[2026-02-14 12:11:56,230: INFO: common: json file saved at: scores.json]


2026/02/14 12:11:56 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:11:57 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:11:57 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:11:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/14 12:11:58 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:11:58 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:11:58 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolvin

[2026-02-14 12:12:01,682: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Mamoon\AppData\Local\Temp\tmpjj19_4gw\model\data\model\assets
[2026-02-14 12:12:02,211: INFO: builder_impl: Assets written to: C:\Users\Mamoon\AppData\Local\Temp\tmpjj19_4gw\model\data\model\assets]


2026/02/14 12:12:22 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:12:22 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:12:23 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:12:23 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:12:24 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:12:24 WARNING mlflow.tracking.request_header.registry: Encountered unexpected error during resolving request headers: list index out of range
2026/02/14 12:12:25 WARNING mlflow.tracking.request_header.regis

🏃 View run bouncy-gull-417 at: https://dagshub.com/mamoonali926568/End-to-End-Chest-Cancer-Classification-System-with-MLOps-DVC-MLflow-CI-CD-AWS-Deployment-.mlflow/#/experiments/0/runs/9459bf574eb9468f9e0241645354b6cd
🧪 View experiment at: https://dagshub.com/mamoonali926568/End-to-End-Chest-Cancer-Classification-System-with-MLOps-DVC-MLflow-CI-CD-AWS-Deployment-.mlflow/#/experiments/0


KeyboardInterrupt: 